# Reading Signup Genius

In [1]:
import pandas as pd 

In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


# %%
import requests 
import pandas as pd
import time 
import numpy as np
import re

print("Reading Timetree website and analyzing data...")

# Reading in Signup Genius URL and data

intl_url = 'https://www.signupgenius.com/go/10c0f4aaeab2fa2fac07-july'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(intl_url, headers=header)

# Finding the right table on the page

dfs = pd.read_html(r.text)
df = dfs[3]
df.rename(columns={0:'Role', 2:'Person'} , inplace = True)
df.drop(columns=1, inplace = True)

# %%
# Cleaning up data - removing double names, signup, slots filled, etc. 

to_remove = {
    '2 of 3 slots filled',
    'All slots filled',
    'Sign Up',
    'O yeah'
}

to_replace = {
    'Joshua Reeves Reeves' : 'Joshua Reeves',
    'Bill Collins Collins' : 'Bill Collins'
}

df['Person'] = df['Person'].str.replace('|'.join(to_remove),"", regex = True)
df['Person'] = df['Person'].str.replace('Joshua Reeves Reeves','Joshua Reeves')
df['Person'] = df['Person'].str.replace('Bill Collins Collins','Bill Collins')
df['Person'] = df['Person'].str.replace('Jesse J Gage','Jesse Gage')

df['Role'] = df['Role'].str.split('(').str[0]

# %%
# Adding emoji for data:attributes:title

emoji = []

#Emojis are below! 

# TOPIC GUITAR = U0001F3B8
# GREET BANJO = U0001FA95
# SECRETARY MICROPHONE = U0001F3A4

for i in df['Role']: 
    if 'Greeter' in i:
        i = "\U0001FA95 "
    elif 'Topic' in i: 
        i = "\U0001F3B8 " 
    elif 'Secretary' in i: 
        i = "\U0001F3A4 "
    emoji.append(i)

df['Emoji'] = emoji

# %%
# Adding verb for data:attributes:title

verb = []

for i in df['Role']: 
    if 'Greeter' in i:
        i = 'greet!'
    elif 'Topic' in i: 
        i = 'brings topic!'
    elif 'Secretary' in i: 
        i = 'Secretarys!'
    verb.append(i)

df['Verb'] = verb
df.head(40)

df['first'] = df['Person'].str.split().str[0]
df['second'] = df['Person'].str.split().str[2]
df['third'] = df['Person'].str.split().str[4]
df = df.fillna('')
df['Combined'] = df['first'] + " " + df['second'] + " " + df['third']
df['Count'] = df['Combined'].str.split().str.len()

df.loc[df['Count'] == 3, 'data:attributes:title'] = df['Emoji'] + df['first'] + ", " + df['second'] + " & " + df['third'] + " " + df['Verb']
df.loc[df['Count'] == 2, 'data:attributes:title'] = df['Emoji'] + df['first'] + " & " + df['second'] + " " + df['Verb']
df.loc[df['Count'] == 1, 'data:attributes:title'] = df['Emoji'] + df['first'] + " " +  df['Verb']

# %%
# Cleaning for export 

df['Day'] = df['Role'].str.split().str[0]
df['Role'] = df['Role'].str.split(" ", 1).str[1]
df.drop(columns={'Person','Verb','first','second','third','Combined','Count','Emoji'} , inplace = True)
df = df[['Day','Role','data:attributes:title']]
df.head()


# In[6]:


script_url = 'https://docs.google.com/document/d/1tVsWegpQ8LR7J20m8sRVA9z-XuKPzDwX7d8yQER-X1c/edit?usp=sharing'
zoom_url = 'https://us04web.zoom.us/j/406536344'
df.head()

url = []

for i in df['Role']: 
    if 'Greeter' in i:
        i = zoom_url
    elif 'Topic' in i: 
        i = zoom_url
    elif 'Secretary' in i: 
        i = script_url
    url.append(i)

df['url'] = url
df['description'] = ''
df.head(40)

Reading Timetree website and analyzing data...


Day           Role           data:attributes:title  \
0      Sunday       Greeter            🪕 Jackie & Jac greet!   
1      Sunday      Secretary             🎤 Randy Secretarys!   
2      Sunday  Topic Bringer           🎸 Ricki brings topic!   
3      Monday       Greeter   🪕 Joshua, Blake & Karen greet!   
4      Monday      Secretary            🎤 JoAnna Secretarys!   
5      Monday  Topic Bringer           🎸 Brett brings topic!   
6     Tuesday       Greeter         🪕 Jackie & Joshua greet!   
7     Tuesday      Secretary            🎤 Jackie Secretarys!   
8     Tuesday  Topic Bringer          🎸 JoAnna brings topic!   
9   Wednesday       Greeter   🪕 Ricki, JoAnna & Karen greet!   
10  Wednesday      Secretary           🎤 Colleen Secretarys!   
11   Thursday       Greeter        🪕 Jackie & Stephie greet!   
12   Thursday      Secretary             🎤 Ricki Secretarys!   
13   Thursday  Topic Bringer           🎸 Jesse brings topic!   
14     Friday       Greeter       🪕 Bill, Jac & Karen greet!   
15     Friday      Secretary               🎤 Jac Secretarys!   
16     Friday  Topic Bringer            🎸 Bill brings topic!   
17   Saturday       Greeter          🪕 Jackie & Blake greet!   
18   Saturday      Secretary           🎤 Caitlin Secretarys!   
19   Saturday  Topic Bringer            🎸 Dana brings topic!   

                                                  url description  
0                 https://us04web.zoom.us/j/406536344              
1   https://docs.google.com/document/d/1tVsWegpQ8L...              
2                 https://us04web.zoom.us/j/406536344              
3                 https://us04web.zoom.us/j/406536344              
4   https://docs.google.com/document/d/1tVsWegpQ8L...              
5                 https://us04web.zoom.us/j/406536344              
6                 https://us04web.zoom.us/j/406536344              
7   https://docs.google.com/document/d/1tVsWegpQ8L...              
8                 https://us04web.zoom.us/j/406536344              
9                 https://us04web.zoom.us/j/406536344              
10  https://docs.google.com/document/d/1tVsWegpQ8L...              
11                https://us04web.zoom.us/j/406536344              
12  https://docs.google.com/document/d/1tVsWegpQ8L...              
13                https://us04web.zoom.us/j/406536344              
14                https://us04web.zoom.us/j/406536344              
15  https://docs.google.com/document/d/1tVsWegpQ8L...              
16                https://us04web.zoom.us/j/406536344              
17                https://us04web.zoom.us/j/406536344              
18  https://docs.google.com/document/d/1tVsWegpQ8L...              
19                https://us04web.zoom.us/j/406536344

# Export File to CSV for Load

In [3]:
# print("Data exporting...")
# df.to_csv(r'/Users/jaccottam/Desktop/Projects/INTL/db/timetree.csv')
# print("Data exported to db Check for file named timetree.csv.")

# Creating Date Tableu for Meeting Population

Use the following table to create dates for the meeting in the future.

In [4]:
def create_date_table(start='2022-08-01', end='2022-08-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["month"] = df.Date.dt.month
    df["day"] = df.Date.dt.day
    df["year"] = df.Date.dt.year
    df["day_of_week"] = df.Date.dt.day_name()
    return df

dates = create_date_table()

dates.head(50)

Date  month  day  year day_of_week
0  2022-08-01      8    1  2022      Monday
1  2022-08-02      8    2  2022     Tuesday
2  2022-08-03      8    3  2022   Wednesday
3  2022-08-04      8    4  2022    Thursday
4  2022-08-05      8    5  2022      Friday
5  2022-08-06      8    6  2022    Saturday
6  2022-08-07      8    7  2022      Sunday
7  2022-08-08      8    8  2022      Monday
8  2022-08-09      8    9  2022     Tuesday
9  2022-08-10      8   10  2022   Wednesday
10 2022-08-11      8   11  2022    Thursday
11 2022-08-12      8   12  2022      Friday
12 2022-08-13      8   13  2022    Saturday
13 2022-08-14      8   14  2022      Sunday
14 2022-08-15      8   15  2022      Monday
15 2022-08-16      8   16  2022     Tuesday
16 2022-08-17      8   17  2022   Wednesday
17 2022-08-18      8   18  2022    Thursday
18 2022-08-19      8   19  2022      Friday
19 2022-08-20      8   20  2022    Saturday
20 2022-08-21      8   21  2022      Sunday
21 2022-08-22      8   22  2022      Monday
22 2022-08-23      8   23  2022     Tuesday
23 2022-08-24      8   24  2022   Wednesday
24 2022-08-25      8   25  2022    Thursday
25 2022-08-26      8   26  2022      Friday
26 2022-08-27      8   27  2022    Saturday
27 2022-08-28      8   28  2022      Sunday
28 2022-08-29      8   29  2022      Monday
29 2022-08-30      8   30  2022     Tuesday
30 2022-08-31      8   31  2022   Wednesday